# FFM Model

## Imports

In [19]:
import random
import numpy as np

from pyspark.sql import types
from pyspark.sql import SQLContext
from pyspark.sql.functions import isnan
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

import time
import numpy as np
import matplotlib.pyplot as plt

In [20]:
from pyspark.sql import SparkSession

app_name = "final_project_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

PWD = !pwd
PWD = PWD[0]

## Load Data

I pushed some parquet files to GitHub to make testing code easier
1. Noah's small example: sample.parquet
2. 100 randomly selected lines from train.txt: smallTrain.parquet
3. 5000 randomly selected lines from train.txt: mediumTrain.parquet

The full is not on GitHub due to its size. But the code for opening train.txt and write it to parquet and save is provided below.

In [8]:
# code for opening train.txt, write and save as parquet
# train_data = spark.read.csv(f"{PWD}/data/train.txt", sep="\t")
# train_data.write.format("parquet").save(f"{PWD}/data/train.parquet")

In [21]:
# select which data to load:
# 1->sample.parquet
# 2->smallTrain.parquet
# 3->mediumTrain.parquet
# 4->train.parquet (full dataset)

DATA_TO_LOAD = 1

In [22]:
if DATA_TO_LOAD == 1:
    train_parquet = spark.read.parquet(f"{PWD}/data/sample.parquet")
    cate_field_start = 2
    cate_field_end = 4
else:
    if DATA_TO_LOAD == 2:
        train_parquet = spark.read.parquet(f"{PWD}/data/smallTrain.parquet")
    elif DATA_TO_LOAD == 3:
        train_parquet = spark.read.parquet(f"{PWD}/data/mediumTrain.parquet")
    else:
        train_parquet = spark.read.parquet(f"{PWD}/data/train.parquet")
    cate_field_start = 14
    cate_field_end = 40
    

In [11]:
## use when on wk08 environment

# sqlContext = SQLContext(sc)
    
# sc = spark.sparkContext
# train_parquet = spark.read.parquet("gs://w261-bucket-hou/final-project/data/train.parquet")
# # train_parquet = spark.read.parquet("gs://w261-bucket-hou/final-project/data/mediumTrain.parquet")
# cate_field_start = 14
# cate_field_end = 40

In [23]:
#rename files and recast integer types on the numeric features

oldColNames = train_parquet.schema.names

train_parquet = train_parquet.withColumn("label", train_parquet["_c0"])
for colNum in range(1,cate_field_start): 
    colName = "_c" + str(colNum)
    train_parquet = train_parquet.withColumn("int_feature_"+ str(colNum), train_parquet[colName].cast(types.IntegerType()))
for colNum in range(cate_field_start,cate_field_end): 
    colName = "_c" + str(colNum)
    train_parquet = train_parquet.withColumn("cate_feature_"+ str(colNum-cate_field_start+1), train_parquet[colName])

#drop the old columns
train_parquet = train_parquet.drop(*oldColNames)

In [24]:
#record feature names by feature type
intFieldNames = [colName for colName, dType in train_parquet.dtypes if dType == 'int']
cateFieldNames = [colName for colName, dType in train_parquet.dtypes if dType == 'string' and colName != 'label']

In [25]:
# view data before feature engineering
train_parquet.show()

+-----+-------------+--------------+--------------+
|label|int_feature_1|cate_feature_1|cate_feature_2|
+-----+-------------+--------------+--------------+
|    1|           10|          ESPN|          Nike|
|    1|           15|          ESPN|          Nike|
|    0|            2|          ESPN|         Gucci|
|    1|           10|          ESPN|        Adidas|
|    1|           10|          ESPN|        Adidas|
|    0|            3|         Vogue|          Nike|
|    1|           20|         Vogue|         Gucci|
|    0|            5|         Vogue|        Adidas|
|    1|           50|           NBC|          Nike|
|    0|            0|           NBC|         Gucci|
|    0|            4|           NBC|        Adidas|
|    0|            4|           NBC|        Adidas|
+-----+-------------+--------------+--------------+



## Feature Engineering

### Categorical Variables

We replace the categorical values having counts less than a certain threshold with a special value `***` to prevent overfitting.

In [26]:
threshold = 4

train_parquet_MD = train_parquet

for col in cateFieldNames:
    valuesToKeep = train_parquet.groupBy(col).count().filter(f"count >= {threshold}").select(col)
    valuesToKeep = valuesToKeep.withColumn("_"+col, train_parquet[col])
    valuesToKeep = valuesToKeep.drop(col)

    train_parquet_MD = train_parquet_MD.join(F.broadcast(valuesToKeep), train_parquet_MD[col] == valuesToKeep["_"+col], 'leftouter')
    train_parquet_MD = train_parquet_MD.withColumn(col, F.when(F.col("_"+col).isNull(), "***").otherwise(F.col("_"+col)))
    train_parquet_MD = train_parquet_MD.drop("_"+col)
    

In [27]:
# view data after the replacement
start = time.time()
train_parquet = train_parquet_MD
train_parquet.show()
print(f'categorical columns processed in {time.time() - start} seconds.')

+-----+-------------+--------------+--------------+
|label|int_feature_1|cate_feature_1|cate_feature_2|
+-----+-------------+--------------+--------------+
|    1|           10|          ESPN|          Nike|
|    1|           15|          ESPN|          Nike|
|    0|            2|          ESPN|           ***|
|    1|           10|          ESPN|        Adidas|
|    1|           10|          ESPN|        Adidas|
|    0|            3|           ***|          Nike|
|    1|           20|           ***|           ***|
|    0|            5|           ***|        Adidas|
|    1|           50|           NBC|          Nike|
|    0|            0|           NBC|           ***|
|    0|            4|           NBC|        Adidas|
|    0|            4|           NBC|        Adidas|
+-----+-------------+--------------+--------------+

categorical columns processed in 1.1284282207489014 seconds.


### Numeric Variables

We take the log of numeric variables

In [28]:
for col in intFieldNames:
    train_parquet = train_parquet.withColumn(col, F.floor(F.log(F.col(col))))

In [29]:
start = time.time()
train_parquet.show()
print(f'... completed job in {time.time() - start} seconds.')

+-----+-------------+--------------+--------------+
|label|int_feature_1|cate_feature_1|cate_feature_2|
+-----+-------------+--------------+--------------+
|    1|            2|          ESPN|          Nike|
|    1|            2|          ESPN|          Nike|
|    0|            0|          ESPN|           ***|
|    1|            2|          ESPN|        Adidas|
|    1|            2|          ESPN|        Adidas|
|    0|            1|           ***|          Nike|
|    1|            2|           ***|           ***|
|    0|            1|           ***|        Adidas|
|    1|            3|           NBC|          Nike|
|    0|         null|           NBC|           ***|
|    0|            1|           NBC|        Adidas|
|    0|            1|           NBC|        Adidas|
+-----+-------------+--------------+--------------+

... completed job in 1.0382089614868164 seconds.


### Hashing

In [30]:
n_features = 100
n_fields = len(intFieldNames) + len(cateFieldNames)

In [31]:
from pyspark.ml.feature import FeatureHasher
hasher = FeatureHasher()
hasher.setCategoricalCols(intFieldNames)
hasher.setNumFeatures(n_features)

for col in intFieldNames + cateFieldNames:
    hasher.setInputCols([col])
    hasher.setOutputCol(col+"_hashed")
    train_parquet = hasher.transform(train_parquet)

In [32]:
start = time.time()
train_parquet.show()
print(f'... completed job in {time.time() - start} seconds.')

+-----+-------------+--------------+--------------+--------------------+---------------------+---------------------+
|label|int_feature_1|cate_feature_1|cate_feature_2|int_feature_1_hashed|cate_feature_1_hashed|cate_feature_2_hashed|
+-----+-------------+--------------+--------------+--------------------+---------------------+---------------------+
|    1|            2|          ESPN|          Nike|    (100,[15],[1.0])|      (100,[7],[1.0])|     (100,[36],[1.0])|
|    1|            2|          ESPN|          Nike|    (100,[15],[1.0])|      (100,[7],[1.0])|     (100,[36],[1.0])|
|    0|            0|          ESPN|           ***|    (100,[38],[1.0])|      (100,[7],[1.0])|     (100,[86],[1.0])|
|    1|            2|          ESPN|        Adidas|    (100,[15],[1.0])|      (100,[7],[1.0])|     (100,[15],[1.0])|
|    1|            2|          ESPN|        Adidas|    (100,[15],[1.0])|      (100,[7],[1.0])|     (100,[15],[1.0])|
|    0|            1|           ***|          Nike|    (100,[15]

In [33]:
hashed_columns = train_parquet.schema.names[-n_fields:]

In [34]:
# build a udf to parse sparse vectors
def parse_sparse_vectors(vector, field_ind):
    if vector.indices.size > 0:
        return f'({field_ind},{vector.indices[0]})'
    else:
        return None

vector_parser = F.udf(parse_sparse_vectors, types.StringType())

In [35]:
for field_ind, col in enumerate(hashed_columns):
    
    train_parquet = train_parquet.withColumn(col, vector_parser(col, F.lit(field_ind)))

train_parquet = train_parquet.drop(*(intFieldNames + cateFieldNames))

In [36]:
start = time.time()
train_parquet.show()
print(f'... completed job in {time.time() - start} seconds.')

+-----+--------------------+---------------------+---------------------+
|label|int_feature_1_hashed|cate_feature_1_hashed|cate_feature_2_hashed|
+-----+--------------------+---------------------+---------------------+
|    1|              (0,15)|                (1,7)|               (2,36)|
|    1|              (0,15)|                (1,7)|               (2,36)|
|    0|              (0,38)|                (1,7)|               (2,86)|
|    1|              (0,15)|                (1,7)|               (2,15)|
|    1|              (0,15)|                (1,7)|               (2,15)|
|    0|              (0,15)|               (1,92)|               (2,36)|
|    1|              (0,15)|               (1,92)|               (2,86)|
|    0|              (0,15)|               (1,92)|               (2,15)|
|    1|              (0,79)|               (1,27)|               (2,36)|
|    0|                null|               (1,27)|               (2,86)|
|    0|              (0,15)|               (1,27)| 

==================================================================================================================================================================================